In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
customer_df = pd.read_excel(r'C:\Users\Asus\Downloads\Shipment.xlsx')
customer_df

,customer,latitude,longitude,demand
0,1,4.3555,113.9777,5
1,2,4.3976,114.0049,8
2,3,4.3163,114.0764,3
3,4,4.3184,113.9932,6
4,5,4.4024,113.9896,5
5,6,4.4142,144.0127,8
6,7,4.4804,144.0734,3
7,8,4.3818,144.2034,6
8,9,4.4935,144.1828,5
9,10,4.4932,144.1322,8


In [3]:
def calculate_euclidean_distance(latitude1, longitude1, latitude2, longitude2):
    return float(np.sqrt((latitude1 - latitude2)**2 + (longitude1 - longitude2)**2))

In [4]:
def generate_random_route(customer_df, vehicle_capacity):
    
    # Shuffle customer sequence
    
    shuffled_customer_df = customer_df.sample(frac=1).reset_index(drop=True)
    
    vehicle_demand = 0
    customer_list = []
    
    for index, row in shuffled_customer_df.iterrows():
        vehicle_demand_temp = vehicle_demand + row['demand']
        if vehicle_demand_temp < vehicle_capacity:
            vehicle_demand += row['demand']
            customer_list.append(row['customer'])

    return customer_list, vehicle_demand

In [5]:
def get_permutation(route_sequence):
    return list(itertools.permutations(route_sequence))

In [6]:
def calculate_route_distance(route, depot_latitude, depot_longitude):
   
    # Calculate distance from depot to first customer

    total_distance = calculate_euclidean_distance(depot_latitude,
                                        depot_longitude,
                                        customer_df.loc[customer_df['customer'] == route[0], 'latitude'].values[0],
                                        customer_df.loc[customer_df['customer'] == route[0], 'longitude'].values[0])
    
    # Calculate distance between customers
    
    for x in route[1:]:
        total_distance = total_distance + calculate_euclidean_distance(customer_df[customer_df['customer'] == x]['latitude'],
                                       customer_df[customer_df['customer'] == x]['longitude'],
                                       customer_df.loc[customer_df['customer'] == route[route.index(x) - 1], 'latitude'].values[0],
                                       customer_df.loc[customer_df['customer'] == route[route.index(x) - 1], 'longitude'].values[0])
    
    # Calculate distance from last customer back to depot
    
    total_distance = total_distance + calculate_euclidean_distance(depot_latitude,
                                        depot_longitude,
                                        customer_df.loc[customer_df['customer'] == route[len(route) - 1], 'latitude'].values[0],
                                        customer_df.loc[customer_df['customer'] == route[len(route) - 1], 'longitude'].values[0])

    return total_distance

In [7]:
def find_best_route_sequence(vehicle_capacity, distance_cost, depot_latitude, depot_longitude):

    best_route_sequence, total_vehicle_demand = generate_random_route(customer_df, vehicle_capacity)
    minimum_distance = calculate_route_distance(best_route_sequence, depot_latitude, depot_longitude)
    route_permutation = get_permutation(best_route_sequence)
    count = 0
    
    for route in route_permutation: 
        route_distance = calculate_route_distance(route, depot_latitude, depot_longitude)
        
        if route_distance < minimum_distance:
            minimum_distance = route_distance
            best_route_sequence = route
            count = 0
        else:
            count = count + 1

        # If there is no new minimum distance after 20 loops, stop the calculation
            
        if count == 20: 
            break
        
    return best_route_sequence, minimum_distance, total_vehicle_demand, minimum_distance*distance_cost

It was observed that 
- If only one iteration is used, only one set of random customer will be used for permutation, other customers will be ignored.
- Thus, to further optimize the solution, 100 iterations are used to take multiple random sets of customters to calculate the minimum cost. (Cons: larger computation cost if there are more customers in the future)

In [8]:
# Inputs

capacity = 30
cost_per_km = 1.5
depot_latitude = 4.4184
depot_logitude = 114.0932

In [9]:
route_list, distance_list, vehicle_demand_list, cost_list = [], [], [], []

for _ in range(100):
    route, distance, vehicle_demand, cost = find_best_route_sequence(capacity, cost_per_km, depot_latitude, depot_logitude)
    route_list.append(route)
    distance_list.append(distance)
    vehicle_demand_list.append(vehicle_demand)
    cost_list.append(cost)

In [10]:
print('Vehicle with Capacity {}, RM {} per km'.format(capacity, cost_per_km))
print('------------------------------------------------')
print('Distance (km): ', min(distance_list))
print('Cost (RM): ', cost_list[distance_list.index(min(distance_list))])
print('Demand: ', int(vehicle_demand_list[distance_list.index(min(distance_list))]))

route_string = 'Depot'

for customer in route_list[distance_list.index(min(distance_list))]:
    route_string = '{} -> {}'.format(route_string, int(customer))

route_string = '{} -> {}'.format(route_string, 'Depot')

print('Route: ', route_string)

Vehicle with Capacity 30, RM 1.5 per km
------------------------------------------------
Distance (km):  0.3820453378700166
Cost (RM):  0.5730680068050249
Demand:  27
Route:  Depot -> 3 -> 4 -> 1 -> 5 -> 2 -> Depot


In [11]:
# Inputs

capacity = 25
cost_per_km = 1.2

In [12]:
route_list, distance_list, vehicle_demand_list, cost_list = [], [], [], []

for _ in range(100):
    route, distance, vehicle_demand, cost = find_best_route_sequence(capacity, cost_per_km, depot_latitude, depot_logitude)
    route_list.append(route)
    distance_list.append(distance)
    vehicle_demand_list.append(vehicle_demand)
    cost_list.append(cost)

In [13]:
print('Vehicle with Capacity {}, RM {} per km'.format(capacity, cost_per_km))
print('------------------------------------------------')
print('Distance (km): ', min(distance_list))
print('Cost (RM): ', cost_list[distance_list.index(min(distance_list))])
print('Demand: ', int(vehicle_demand_list[distance_list.index(min(distance_list))]))

route_string = 'Depot'

for customer in route_list[distance_list.index(min(distance_list))]:
    route_string = '{} -> {}'.format(route_string, int(customer))

route_string = '{} -> {}'.format(route_string, 'Depot')

print('Route: ', route_string)

Vehicle with Capacity 25, RM 1.2 per km
------------------------------------------------
Distance (km):  0.3775285795890166
Cost (RM):  0.4530342955068199
Demand:  22
Route:  Depot -> 3 -> 4 -> 5 -> 2 -> Depot
